##### Criação de uma tabela de gastos com o método SQL UNION
- union()	Quando os DataFrames têm mesmo esquema e mesma ordem de colunas
- unionByName()	Quando os DataFrames têm mesmo esquema mas ordem diferente
- unionByName(..., allowMissingColumns=True)	Quando os DataFrames têm colunas parcialmente diferentes

In [0]:
df_bradesco = spark.table('personalfinance.silver.gastos_bradesco')

In [0]:
df_btg = spark.table('personalfinance.silver.gastos_btg')

In [0]:
df_boletos = spark.table('personalfinance.silver.gastos_boletos')

In [0]:
from pyspark.sql.functions import col, when

In [0]:
print(df_bradesco.count() + df_btg.count() + df_boletos.count())

In [0]:
df_union = df_bradesco.unionByName(df_btg, allowMissingColumns=True) \
                    .unionByName(df_boletos, allowMissingColumns=True)

In [0]:
mapeamento_neo = [
    # Transporte e combustível
    ("TRANSPORTE", r"ESTAC|POSTO|UBER|COMBUST|ECORODO|PNEUS|PONTE\sNOVA"),
    # Compra imóvel
    ("IMOVEL", r'CANOPUS'),
    # Contas
    ("CONTAS", r'VIVO'),
    # Assinaturas e serviços recorrentes
    ("ASSINATURA", r"SPOTIFY|APPLE|ANUIDADE|DISNEY"),
    # Compras e varejo
    ("COMPRAS", r"TRACK|AMAZON|RENNER|HAVAIANAS|LINDT|LOCCITANE|WORLD WINE|MERCADOLIVRE|CASAS\sBAHIA|MERCADO\sLIVRE|DUTY|ARTE|LIVEBH"),
    # Viagem e hospedagem
    ("VIAGEM", r"HOTEL|AIRLINE|CORCOVA|BOOKING|PASSAGEM|TRIP|RADISSON|KL\sINTER|LOUNGEKEY|AIR|HEINEKEN|REGIOJET|HOLAFLY|GET\sYOUR\sGUIDE|VUELO"),
    # Saúde e cuidados pessoais
    ("SAUDE & CUIDADOS PESSOAIS", r"DROGA|RAIA|ARAUJO|FUNCIONARIOS|ISAURA|FARMACIA|GILBERTO\sVIANNA|CORPO"),
    # Transporte e combustível
    ("TRANSPORTE", r"ESTAC|POSTO|UBER|COMBUST|ECORODO|PNEUS|PONTE\sNOVA"),
    # Seguros
    ("SEGURO", r"YELUMSEG|YELUM"),
    # Educação e serviços
    ("ESTUDOS & CAPACITACAO", r"UDEMY|MICROSOFT|AZURE|KRYTERION|ALURA|ENHANCV|LINKED|IBMEC"),
    # Taxas e encargos financeiros
    ("FINANCEIRO", r"IOF|PAG BOLETO|SALDO ANTERIOR"),
    # Lazer, bares e entretenimento
    ("LAZER", r"ZIG|ALMANAQUE|CINE|BERILO|BROU|BUTIQUIM|CERVEJARIA|INGRESSO|SECRETO|EVENTS|CASACOR|BAR|MARU|K08|KEI|PATRIA|BENVINDO|68|MEEP|NOLITA|INGRESSE|SAKANA|PELLEGRINO|SEU\sBIAS"),
    # Alimentação (restaurantes, lanchonetes, delivery, padarias)
    ("ALIMENTACAO", r"ALIM|PAO|RANCHO|EPA|SUPER|POMAR|BOCA|AMBROZINI|ASSA|SORVETE|FORNO|IFOOD|RESTAUR|LANCH|PIZZA|COZINHA|SUCO|EMPORIO|MEGAMATTE|OAKBERRY|MACELLERIA|GASTRO|PANIFICADORA|SALGADOS|PASTEL|ALEMAO|SODEXO|ACAI|TAKE|GRAAL|LULLO|MARILIA|KOMBI|ENGENHEIROS|SKYASIA|PIZZ|JAPO|SABOR|PORKS|BURGUER|IFD|KFC"),
]

In [0]:
condicao = None

for classe, regex in mapeamento_neo:
    if regex:  # ignora regex vazio
        if condicao is None:
            condicao = when(col("descricao").rlike(regex), classe)
        else:
            condicao = condicao.when(col("descricao").rlike(regex), classe)

df_union = df_union.withColumn("classe_gasto", condicao.otherwise("OUTROS"))

In [0]:
df_union.createOrReplaceTempView('tv_gastos')

In [0]:
%sql
SELECT classe_gasto, SUM(valor_brl) AS total_valor, COUNT(descricao) AS total_compras
  FROM tv_gastos
  GROUP BY classe_gasto
  ORDER BY total_valor DESC;

In [0]:
df_union.write \
    .mode('overwrite') \
    .format('delta') \
    .option('overwriteSchema', True) \
    .saveAsTable('personalfinance.silver.gastos')